In [ ]:
import pandas as pd

### Read saved data

In [ ]:
df = pd.read_csv("data.csv",index_col=0)
df.columns

### Calculate features

In [ ]:
# get DATE, PRICE, 120d_RPS, E/P, B/P, EBITDA/EV, REVENUE/EV, Market Cap (MB.), ValueTrade
# VC Damodaran, that combines P/E, P/B, EV/Sales, EV/EBITDA and P/FCF

In [ ]:
screener_df = pd.DataFrame(index=df.index.copy())
screener_df['Price'] = df['prior']
screener_df['120d_RPS'] = df['sixMonthPercentChange']
screener_df['E/P'] = 1/df['peRatio']
screener_df['B/P'] = 1/df['pbRatio']
screener_df['EBITDA/EV'] = 1/df['evToEbitdaRatio']
screener_df['REVENUE/EV'] = (1/df['peRatio'])*df['marketCap']/(df['Net Profit Margin (%)']/100)/df['ev']
screener_df['TradeValue'] = df['ytdAverageValue']
screener_df['ValueComposite'] = (screener_df['E/P'].rank(pct=True, ascending=False) + screener_df['B/P'].rank(pct=True, ascending=False) + screener_df['EBITDA/EV'].rank(
        pct=True, ascending=False) + screener_df['REVENUE/EV'].rank(pct=True, ascending=False)).rank(pct=True, ascending=False)
screener_df

In [ ]:
screener_df = screener_df[screener_df['ValueComposite'] > 0.85].sort_values('120d_RPS', ascending=False)
screener_df

In [ ]:
screener_df.index.name = 'Symbol'
screener_df.to_excel("result.xlsx")

### Manual get portfolio

In [ ]:
portfolio = pd.read_excel('portfolio.xlsx')
portfolio

In [ ]:
line_available = 0

### New portfolio plan

In [ ]:
screener_result = pd.read_excel('result.xlsx')
screener_result

In [ ]:
account_value = line_available + portfolio['Value'].sum()
target_value_per_stock = account_value/20
target_value_per_stock

In [ ]:
target_portfolio = screener_result[screener_result['TradeValue']>=10000000].iloc[:20].copy()
target_portfolio['Target'] = target_portfolio.apply(lambda x: (target_value_per_stock/x['Price'])-(target_value_per_stock/x['Price'])%100,axis=1)
target_portfolio['Value'] = target_portfolio['Target']*target_portfolio['Price']
target_portfolio

### Compare portfoilo / To do list

In [ ]:
days = 2
todo = pd.merge(portfolio[['Symbol','Available']],target_portfolio[['Symbol','Price','Target']],how='outer',on='Symbol',suffixes=('_current','_target')).fillna(0)
todo['Buy(-Sell)'] = todo['Target']-todo['Available']
todo = todo.sort_values('Buy(-Sell)')
todo['Buy(-Sell)_Chunk'] = todo['Buy(-Sell)']/days - (todo['Buy(-Sell)']/days)%100
todo['Buy(-Sell)_Chunk_Value'] = todo['Buy(-Sell)_Chunk']*todo['Price']
todo

In [ ]:
todo.to_excel('todo.xlsx',index=False)